- Usa ARIMA si tienes una serie temporal no estacional que presenta tendencia.
- Usa SARIMA si tienes datos estacionales con tendencias.
- Considera modelos derivados (ARIMAX, SARIMAX) si tienes variables externas que podrían afectar la serie temporal que estás modelando.

In [428]:
# Librerías
# ======================================================================================
import numpy as np
import pandas as pd
from io import StringIO
import contextlib
import re
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')

# pmdarima
import pmdarima
from pmdarima import ARIMA
from pmdarima import auto_arima

# statsmodels
import statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from skforecast.model_selection import bayesian_search_forecaster

# skforecast
import skforecast
from skforecast.datasets import fetch_dataset
from skforecast.plot import set_dark_theme
from skforecast.Sarimax import Sarimax
from skforecast.ForecasterSarimax import ForecasterSarimax
from skforecast.model_selection_sarimax import backtesting_sarimax
from skforecast.model_selection_sarimax import grid_search_sarimax
from sklearn.model_selection import train_test_split
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from lightgbm import LGBMRegressor
from skforecast.model_selection import backtesting_forecaster
import plotly.graph_objects as go
from sklearn.preprocessing import OneHotEncoder

from astral import LocationInfo
from astral.sun import sun
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer

import warnings
warnings.filterwarnings('once')

color = '\033[1m\033[38;5;208m' 
print(f"{color}Versión skforecast: {skforecast.__version__}")
print(f"{color}Versión pdarima: {pmdarima.__version__}")
print(f"{color}Versión statsmodels: {statsmodels.__version__}")
print(f"{color}Versión pandas: {pd.__version__}")
print(f"{color}Versión numpy: {np.__version__}")

Versión skforecast: 0.13.0
Versión pdarima: 2.0.4
Versión statsmodels: 0.14.3
Versión pandas: 2.2.3
Versión numpy: 1.26.4


In [403]:
# Cargar el dataset
df = pd.read_excel('atm_historical_data_with_features.xlsx')
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S')
df = df.sort_values(by=['ATM_ID', 'Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3580 entries, 0 to 3579
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      3580 non-null   datetime64[ns]
 1   ATM_ID                    3580 non-null   int64         
 2   Agency                    3580 non-null   object        
 3   Day_of_Week               3580 non-null   object        
 4   Soles_Withdrawn           3580 non-null   int64         
 5   Dollars_Withdrawn         3580 non-null   int64         
 6   Event                     2698 non-null   object        
 7   Holiday                   3580 non-null   int64         
 8   Supply_Cost_Fixed         3580 non-null   int64         
 9   Supply_Cost_Variable      3580 non-null   float64       
 10  ATM_Max_Capacity_Soles    3580 non-null   int64         
 11  ATM_Max_Capacity_Dollars  3580 non-null   int64         
 12  Opportunity_Cost    

In [404]:
# Comprobar la presencia de valores nulos
print(df.isnull().sum())

Date                          0
ATM_ID                        0
Agency                        0
Day_of_Week                   0
Soles_Withdrawn               0
Dollars_Withdrawn             0
Event                       882
Holiday                       0
Supply_Cost_Fixed             0
Supply_Cost_Variable          0
ATM_Max_Capacity_Soles        0
ATM_Max_Capacity_Dollars      0
Opportunity_Cost              0
Soles_Withdrawn_Lag1          0
Dollars_Withdrawn_Lag1        0
Soles_Withdrawn_Lag7          0
Dollars_Withdrawn_Lag7        0
Soles_Moving_Avg_7            0
Dollars_Moving_Avg_7          0
Total_Withdrawn               0
Total_Withdrawn_Lag1          0
Total_Withdrawn_Lag7          0
Total_Moving_Avg_7            0
dtype: int64


In [405]:
# Seleccionar las columnas de interés para el modelado
features = [
    'Soles_Withdrawn_Lag1', 'Dollars_Withdrawn_Lag1',
    'Soles_Withdrawn_Lag7', 'Dollars_Withdrawn_Lag7',
    'Soles_Moving_Avg_7', 'Dollars_Moving_Avg_7',
    'Total_Withdrawn_Lag1', 'Total_Withdrawn_Lag7',
    'Total_Moving_Avg_7', 'Holiday'
]

In [406]:
X = df[features]
y_soles = df['Soles_Withdrawn']
y_dollars = df['Dollars_Withdrawn']

# Calcular el índice de división
split_index = int(len(df) * 0.8)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]
y_train_soles = y_soles.iloc[:split_index]
y_test_soles = y_soles.iloc[split_index:]
y_train_dollars = y_dollars.iloc[:split_index]
y_test_dollars = y_dollars.iloc[split_index:]

# Verificar las divisiones
print(f"Fechas train : {df['Date'].iloc[0]} --- {df['Date'].iloc[split_index - 1]}")
print(f"Fechas test  : {df['Date'].iloc[split_index]} --- {df['Date'].iloc[-1]}")

Fechas train : 2023-01-08 00:00:00 --- 2023-12-31 00:00:00
Fechas test  : 2023-01-08 00:00:00 --- 2023-12-31 00:00:00


In [407]:
X_train

,Soles_Withdrawn_Lag1,Dollars_Withdrawn_Lag1,Soles_Withdrawn_Lag7,Dollars_Withdrawn_Lag7,Soles_Moving_Avg_7,Dollars_Moving_Avg_7,Total_Withdrawn_Lag1,Total_Withdrawn_Lag7,Total_Moving_Avg_7,Holiday
0,25450,1524,40469,10437,26518.571429,7486.142857,26974,50906,34004.714286,0
1,20475,8563,34857,13678,25699.428571,7445.571429,29038,48535,33145.000000,1
2,29123,13394,18829,4531,25218.000000,7109.000000,42517,23360,32327.000000,0
3,15459,2175,41844,11107,26853.000000,7068.571429,17634,52951,33921.571429,1
4,53289,10824,17652,6085,28689.142857,6721.428571,64113,23737,35410.571429,0
...,...,...,...,...,...,...,...,...,...,...
2859,27042,10813,20703,7492,26902.714286,9781.428571,37855,28195,36684.142857,1
2860,24462,6664,26812,10636,27107.142857,10519.857143,31126,37448,37627.000000,1
2861,28243,15805,30252,9412,27268.428571,10185.285714,44048,39664,37453.714286,0
2862,31381,7070,33754,22873,25140.142857,9408.000000,38451,56627,34548.142857,0


In [408]:
# Crear el forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
    regressor = LGBMRegressor(random_state=15926, verbose=-1),
    lags      = 24
)
# Entrenar el forecaster
# ==============================================================================
forecaster.fit(y_train_soles)
forecaster

ForecasterAutoreg 
Regressor: LGBMRegressor(random_state=15926, verbose=-1) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
Transformer for y: None 
Transformer for exog: None 
Window size: 24 
Weight function included: False 
Differentiation order: None 
Exogenous included: False 
Exogenous variables names: None 
Training range: [0, 2863] 
Training index type: RangeIndex 
Training index frequency: 1 
Regressor parameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 31, 'objective': None, 'random_state': 15926, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1} 
fit_kwargs: {} 
Creation date: 2024-09-26 21:47:30 
Last fit date: 2024-09-26 21:47:30 
Skforecast version

In [409]:
forecaster.predict(steps=10)

2864    32517.612866
2865    26751.769588
2866    36072.160216
2867    33955.208851
2868    32709.065888
2869    35492.080716
2870    34404.254420
2871    33296.884189
2872    32385.590668
2873    34026.697754
Name: pred, dtype: float64

In [410]:
# Backtest del modelo con los datos de test
# ==============================================================================
metrica, predicciones = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = y_soles,
    steps              = 36,
    metric             = 'mean_absolute_error',
    initial_train_size = len(y_train_soles),
    refit              = False,
    n_jobs             = 'auto',
    verbose            = True, # Cambiar a False para mostrar menos información
    show_progress      = True
)
predicciones.head()  # Muestra las primeras filas de las predicciones

Information of backtesting process
----------------------------------
Number of observations used for initial training: 2864
Number of observations used for backtesting: 716
    Number of folds: 20
    Number skipped folds: 0 
    Number of steps per fold: 36
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 32 observations.

Fold: 0
    Training:   0 -- 2863  (n=2864)
    Validation: 2864 -- 2899  (n=36)
Fold: 1
    Training:   No training in this fold
    Validation: 2900 -- 2935  (n=36)
Fold: 2
    Training:   No training in this fold
    Validation: 2936 -- 2971  (n=36)
Fold: 3
    Training:   No training in this fold
    Validation: 2972 -- 3007  (n=36)
Fold: 4
    Training:   No training in this fold
    Validation: 3008 -- 3043  (n=36)
Fold: 5
    Training:   No training in this fold
    Validation: 3044 -- 3079  (n=36)
Fold: 6
    Training:   No training in this fold
    Validation: 3080 -- 3115  (n=36)
Fold: 7
    Tr

  0%|          | 0/20 [00:00<?, ?it/s]

,pred
2864,32517.612866
2865,26751.769588
2866,36072.160216
2867,33955.208851
2868,32709.065888


In [411]:
metrica

,mean_absolute_error
0,9388.172281


In [412]:
# Búsqueda de hiperparámetros
# ==============================================================================
# Lags candidatos
lags_grid = [48, 72, (1, 2, 3, 23, 24, 25, 167, 168, 169)]

# Espacio de búsqueda de hiperparámetros
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

resultados_busqueda, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = y_soles, # Datos test no incluidos
    search_space       = search_space,
    steps              = 36,
    refit              = False,
    metric             = 'mean_absolute_error',
    initial_train_size = len(y_train_soles),
    fixed_train_size   = False,
    n_trials           = 20, # Aumentar para una búsqueda más exhaustiva
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72] 
  Parameters: {'n_estimators': 400, 'max_depth': 10, 'min_data_in_leaf': 488, 'learning_rate': 0.010018922062789044, 'feature_fraction': 0.6, 'max_bin': 50, 'reg_alpha': 1.0, 'reg_lambda': 0.0}
  Backtesting metric: 9337.955158390336



In [413]:
# Resultados de la búsqueda
# ==============================================================================
resultados_busqueda.head(10)

,lags,params,mean_absolute_error,n_estimators,max_depth,min_data_in_leaf,learning_rate,feature_fraction,max_bin,reg_alpha,reg_lambda
12,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 400, 'max_depth': 10, 'min_da...",9337.955158,400.0,10.0,488.0,0.010019,0.6,50.0,1.0,0.0
10,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 400, 'max_depth': 10, 'min_da...",9339.697471,400.0,10.0,492.0,0.013950,0.5,50.0,0.9,0.2
11,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 400, 'max_depth': 10, 'min_da...",9340.326451,400.0,10.0,483.0,0.014988,0.5,50.0,0.9,0.2
14,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'n_estimators': 400, 'max_depth': 9, 'min_dat...",9417.750691,400.0,9.0,435.0,0.012421,0.6,50.0,0.8,0.2
18,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 500, 'max_depth': 8, 'min_dat...",9480.466557,500.0,8.0,493.0,0.037503,0.7,75.0,0.8,0.3
13,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 400, 'max_depth': 10, 'min_da...",9548.107969,400.0,10.0,487.0,0.092932,0.6,50.0,0.9,0.0
9,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 700, 'max_depth': 7, 'min_dat...",9564.788658,700.0,7.0,342.0,0.061895,0.5,100.0,0.7,0.9
16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 500, 'max_depth': 9, 'min_dat...",9643.117777,500.0,9.0,253.0,0.105588,0.6,75.0,0.8,0.3
7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 500, 'max_depth': 4, 'min_dat...",9894.434248,500.0,4.0,356.0,0.166196,0.9,150.0,0.4,1.0
17,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 800, 'max_depth': 9, 'min_dat...",9910.828091,800.0,9.0,429.0,0.120804,0.5,125.0,1.0,0.0


In [414]:
# Mejor modelo
# ==============================================================================
forecaster

ForecasterAutoreg 
Regressor: LGBMRegressor(feature_fraction=0.6, learning_rate=0.010018922062789044,
              max_bin=50, max_depth=10, min_data_in_leaf=488, n_estimators=400,
              random_state=15926, reg_alpha=1.0, verbose=-1) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72] 
Transformer for y: None 
Transformer for exog: None 
Window size: 72 
Weight function included: False 
Differentiation order: None 
Exogenous included: False 
Exogenous variables names: None 
Training range: [0, 3579] 
Training index type: RangeIndex 
Training index frequency: 1 
Regressor parameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.010018922062789044, 'max_depth': 10, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain':

In [415]:
# Backtest modelo final con datos de test
# ==============================================================================
metrica, predicciones = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = y_soles,
    steps              = 36,
    metric             = 'mean_absolute_error',
    initial_train_size = len(y_train_soles),
    refit              = False,
    n_jobs             = 'auto',
    verbose            = True, # Cambiar a False para mostrar menos información
    show_progress      = True
)
display(metrica)
predicciones.head()

Information of backtesting process
----------------------------------
Number of observations used for initial training: 2864
Number of observations used for backtesting: 716
    Number of folds: 20
    Number skipped folds: 0 
    Number of steps per fold: 36
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 32 observations.

Fold: 0
    Training:   0 -- 2863  (n=2864)
    Validation: 2864 -- 2899  (n=36)
Fold: 1
    Training:   No training in this fold
    Validation: 2900 -- 2935  (n=36)
Fold: 2
    Training:   No training in this fold
    Validation: 2936 -- 2971  (n=36)
Fold: 3
    Training:   No training in this fold
    Validation: 2972 -- 3007  (n=36)
Fold: 4
    Training:   No training in this fold
    Validation: 3008 -- 3043  (n=36)
Fold: 5
    Training:   No training in this fold
    Validation: 3044 -- 3079  (n=36)
Fold: 6
    Training:   No training in this fold
    Validation: 3080 -- 3115  (n=36)
Fold: 7
    Tr

  0%|          | 0/20 [00:00<?, ?it/s]

,mean_absolute_error
0,9337.955158


,pred
2864,34277.705225
2865,31200.647701
2866,32182.154552
2867,32734.188566
2868,31474.386286


In [416]:
# Gráfico predicciones vs valor real
# ======================================================================================
fig = go.Figure()
trace1 = go.Scatter(x=y_test_soles.index, y=y_test_soles, name="test", mode="lines")
trace2 = go.Scatter(x=predicciones.index, y=predicciones['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Predicción vs valor real en los datos de test",
    xaxis_title="Fecha",
    yaxis_title="Predicción",
    width=800,
    height=350,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="left",
        x=0.001
    )
)
fig.show()

In [417]:
import pandas as pd
import numpy as np
from astral.sun import sun
from astral.geocoder import database, lookup
from astral import LocationInfo

# Variables basadas en el calendario
# ==============================================================================
variables_calendario = pd.DataFrame(df['Date'])
variables_calendario['mes'] = variables_calendario['Date'].dt.month
variables_calendario['semana_anyo'] = variables_calendario['Date'].dt.isocalendar().week
variables_calendario['dia_semana'] = variables_calendario['Date'].dt.day_of_week + 1
variables_calendario['day_of_Year'] = variables_calendario['Date'].dt.dayofyear
variables_calendario['hora_dia'] = variables_calendario['Date'].dt.hour + 1

# Variables basadas en la luz solar
# ==============================================================================
location = LocationInfo(
    name='Washington DC',
    region='USA',
    timezone='US/Eastern',
    latitude=40.516666666666666,
    longitude=-77.03333333333333
)

# Asegurarse de iterar sobre las fechas reales
hora_amanecer = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunrise'].hour
    for date in variables_calendario['Date']
]
hora_anochecer = [
    sun(location.observer, date=date, tzinfo=location.timezone)['sunset'].hour
    for date in variables_calendario['Date']
]

variables_solares = pd.DataFrame({
    'hora_amanecer': hora_amanecer,
    'hora_anochecer': hora_anochecer
}, index=variables_calendario.index)

variables_solares['horas_luz_solar'] = (
    variables_solares['hora_anochecer'] - variables_solares['hora_amanecer']
)
variables_solares["es_de_dia"] = np.where(
    (df['Date'].dt.hour >= variables_solares["hora_amanecer"])
    & (df['Date'].dt.hour < variables_solares["hora_anochecer"]),
    1,
    0,
)

# Merge all exogenous variables
# ==============================================================================
variables_exogenas = pd.concat([
    variables_calendario,
    variables_solares,
], axis=1)

variables_exogenas.head(4)

,Date,mes,semana_anyo,dia_semana,day_of_Year,hora_dia,hora_amanecer,hora_anochecer,horas_luz_solar,es_de_dia
0,2023-01-08,1,1,7,8,1,7,16,9,0
1,2023-01-09,1,2,1,9,1,7,16,9,0
2,2023-01-10,1,2,2,10,1,7,17,10,0
3,2023-01-11,1,2,3,11,1,7,17,10,0


In [418]:
# Codificación cíclica de las variables de calendario y luz solar
# ==============================================================================
def codificacion_ciclica(datos: pd.Series, longitud_ciclo: int) -> pd.DataFrame:
    """
    Codifica una variable cíclica con dos nuevas variables: seno y coseno.
    Se asume que el valor mínimo de la variable es 0. El valor máximo de la
    variable se pasa como argumento.
      
    Parameters
    ----------
    datos : pd.Series
        Serie con la variable a codificar.
    longitud_ciclo : int
        La longitud del ciclo. Por ejemplo, 12 para meses, 24 para horas, etc.
        Este valor se utiliza para calcular el ángulo del seno y coseno.

    Returns
    -------
    resultado : pd.DataFrame
        Dataframe con las dos nuevas características: seno y coseno.

    """

    seno = np.sin(2 * np.pi * datos/longitud_ciclo)
    coseno = np.cos(2 * np.pi * datos/longitud_ciclo)
    resultado =  pd.DataFrame({
                  f"{datos.name}_seno": seno,
                  f"{datos.name}_coseno": coseno
              })

    return resultado


mes_encoded = codificacion_ciclica(variables_exogenas['mes'], longitud_ciclo=12)
semana_anyo_encoded = codificacion_ciclica(variables_exogenas['semana_anyo'], longitud_ciclo=52)
dia_semana_encoded = codificacion_ciclica(variables_exogenas['dia_semana'], longitud_ciclo=7)
hora_amanecer_encoded = codificacion_ciclica(variables_exogenas['hora_amanecer'], longitud_ciclo=24)
hora_anochecer_encoded = codificacion_ciclica(variables_exogenas['hora_anochecer'], longitud_ciclo=24)
hora_dia_encoded = codificacion_ciclica(variables_exogenas['hora_dia'], longitud_ciclo=24)

variables_ciclicas = pd.concat([
                            mes_encoded,
                            semana_anyo_encoded,
                            dia_semana_encoded,
                            hora_dia_encoded,
                            hora_amanecer_encoded,
                            hora_anochecer_encoded
                        ], axis=1)  

variables_exogenas = pd.concat([variables_exogenas, variables_ciclicas], axis=1)
variables_exogenas.head(3)

,Date,mes,semana_anyo,dia_semana,day_of_Year,hora_dia,hora_amanecer,hora_anochecer,horas_luz_solar,es_de_dia,...,semana_anyo_seno,semana_anyo_coseno,dia_semana_seno,dia_semana_coseno,hora_dia_seno,hora_dia_coseno,hora_amanecer_seno,hora_amanecer_coseno,hora_anochecer_seno,hora_anochecer_coseno
0,2023-01-08,1,1,7,8,1,7,16,9,0,...,0.120537,0.992709,-2.449294e-16,1.000000,0.258819,0.965926,0.965926,-0.258819,-0.866025,-0.500000
1,2023-01-09,1,2,1,9,1,7,16,9,0,...,0.239316,0.970942,7.818315e-01,0.623490,0.258819,0.965926,0.965926,-0.258819,-0.866025,-0.500000
2,2023-01-10,1,2,2,10,1,7,17,10,0,...,0.239316,0.970942,9.749279e-01,-0.222521,0.258819,0.965926,0.965926,-0.258819,-0.965926,-0.258819


In [419]:
print(variables_exogenas.columns)

Index(['Date', 'mes', 'semana_anyo', 'dia_semana', 'day_of_Year', 'hora_dia',
       'hora_amanecer', 'hora_anochecer', 'horas_luz_solar', 'es_de_dia',
       'mes_seno', 'mes_coseno', 'semana_anyo_seno', 'semana_anyo_coseno',
       'dia_semana_seno', 'dia_semana_coseno', 'hora_dia_seno',
       'hora_dia_coseno', 'hora_amanecer_seno', 'hora_amanecer_coseno',
       'hora_anochecer_seno', 'hora_anochecer_coseno'],
      dtype='object')


In [420]:
# Interacción entre variables exógenas
# ==============================================================================
transformer_poly = PolynomialFeatures(
    degree           = 2,
    interaction_only = True,
    include_bias     = False
).set_output(transform="pandas")

poly_cols = [
    'mes_seno',
    'mes_coseno',
    'semana_anyo_seno',
    'semana_anyo_coseno',
    'dia_semana_seno',
    'dia_semana_coseno',
    'hora_dia_seno',
    'hora_dia_coseno',
    'hora_amanecer_seno',
    'hora_amanecer_coseno',
    'hora_anochecer_seno',
    'hora_anochecer_coseno',
    'horas_luz_solar',
    'es_de_dia',
]

variables_poly = transformer_poly.fit_transform(variables_exogenas[poly_cols].dropna())
variables_poly = variables_poly.drop(columns=poly_cols)
variables_poly.columns = [f"poly_{col}" for col in variables_poly.columns]
variables_poly.columns = variables_poly.columns.str.replace(" ", "__")
variables_exogenas = pd.concat([variables_exogenas, variables_poly], axis=1)
variables_exogenas.head(3)

,Date,mes,semana_anyo,dia_semana,day_of_Year,hora_dia,hora_amanecer,hora_anochecer,horas_luz_solar,es_de_dia,...,poly_hora_amanecer_coseno__hora_anochecer_seno,poly_hora_amanecer_coseno__hora_anochecer_coseno,poly_hora_amanecer_coseno__horas_luz_solar,poly_hora_amanecer_coseno__es_de_dia,poly_hora_anochecer_seno__hora_anochecer_coseno,poly_hora_anochecer_seno__horas_luz_solar,poly_hora_anochecer_seno__es_de_dia,poly_hora_anochecer_coseno__horas_luz_solar,poly_hora_anochecer_coseno__es_de_dia,poly_horas_luz_solar__es_de_dia
0,2023-01-08,1,1,7,8,1,7,16,9,0,...,0.224144,0.129410,-2.329371,-0.0,0.433013,-7.794229,-0.0,-4.50000,-0.0,0.0
1,2023-01-09,1,2,1,9,1,7,16,9,0,...,0.224144,0.129410,-2.329371,-0.0,0.433013,-7.794229,-0.0,-4.50000,-0.0,0.0
2,2023-01-10,1,2,2,10,1,7,17,10,0,...,0.250000,0.066987,-2.588190,-0.0,0.250000,-9.659258,-0.0,-2.58819,-0.0,0.0


In [421]:
# Codificación cíclica de las variables de calendario y luz solar
# ==============================================================================
def codificacion_ciclica(datos: pd.Series, longitud_ciclo: int) -> pd.DataFrame:
    """
    Codifica una variable cíclica con dos nuevas variables: seno y coseno.
    Se asume que el valor mínimo de la variable es 0. El valor máximo de la
    variable se pasa como argumento.
      
    Parameters
    ----------
    datos : pd.Series
        Serie con la variable a codificar.
    longitud_ciclo : int
        La longitud del ciclo. Por ejemplo, 12 para meses, 24 para horas, etc.
        Este valor se utiliza para calcular el ángulo del seno y coseno.

    Returns
    -------
    resultado : pd.DataFrame
        Dataframe con las dos nuevas características: seno y coseno.

    """

    seno = np.sin(2 * np.pi * datos/longitud_ciclo)
    coseno = np.cos(2 * np.pi * datos/longitud_ciclo)
    resultado =  pd.DataFrame({
                  f"{datos.name}_seno": seno,
                  f"{datos.name}_coseno": coseno
              })

    return resultado


mes_encoded = codificacion_ciclica(variables_exogenas['mes'], longitud_ciclo=12)
semana_anyo_encoded = codificacion_ciclica(variables_exogenas['semana_anyo'], longitud_ciclo=52)
dia_semana_encoded = codificacion_ciclica(variables_exogenas['dia_semana'], longitud_ciclo=7)
hora_dia_encoded = codificacion_ciclica(variables_exogenas['hora_dia'], longitud_ciclo=24)
hora_amanecer_encoded = codificacion_ciclica(variables_exogenas['hora_amanecer'], longitud_ciclo=24)
hora_anochecer_encoded = codificacion_ciclica(variables_exogenas['hora_anochecer'], longitud_ciclo=24)

variables_ciclicas = pd.concat([
                            mes_encoded,
                            semana_anyo_encoded,
                            dia_semana_encoded,
                            hora_dia_encoded,
                            hora_amanecer_encoded,
                            hora_anochecer_encoded
                        ], axis=1)  

variables_exogenas = pd.concat([variables_exogenas, variables_ciclicas], axis=1)
variables_exogenas.head(3)

,Date,mes,semana_anyo,dia_semana,day_of_Year,hora_dia,hora_amanecer,hora_anochecer,horas_luz_solar,es_de_dia,...,semana_anyo_seno,semana_anyo_coseno,dia_semana_seno,dia_semana_coseno,hora_dia_seno,hora_dia_coseno,hora_amanecer_seno,hora_amanecer_coseno,hora_anochecer_seno,hora_anochecer_coseno
0,2023-01-08,1,1,7,8,1,7,16,9,0,...,0.120537,0.992709,-2.449294e-16,1.000000,0.258819,0.965926,0.965926,-0.258819,-0.866025,-0.500000
1,2023-01-09,1,2,1,9,1,7,16,9,0,...,0.239316,0.970942,7.818315e-01,0.623490,0.258819,0.965926,0.965926,-0.258819,-0.866025,-0.500000
2,2023-01-10,1,2,2,10,1,7,17,10,0,...,0.239316,0.970942,9.749279e-01,-0.222521,0.258819,0.965926,0.965926,-0.258819,-0.965926,-0.258819


In [422]:
# Interacción entre variables exógenas
# ==============================================================================
transformer_poly = PolynomialFeatures(
    degree           = 2,
    interaction_only = True,
    include_bias     = False
).set_output(transform="pandas")

poly_cols = [
    'mes_seno',
    'mes_coseno',
    'semana_anyo_seno',
    'semana_anyo_coseno',
    'dia_semana_seno',
    'dia_semana_coseno',
    'hora_dia_seno',
    'hora_dia_coseno',
    'hora_amanecer_seno',
    'hora_amanecer_coseno',
    'hora_anochecer_seno',
    'hora_anochecer_coseno',
    'horas_luz_solar',
    'es_de_dia',
]

variables_poly = transformer_poly.fit_transform(variables_exogenas[poly_cols].dropna())
variables_poly = variables_poly.drop(columns=poly_cols)
variables_poly.columns = [f"poly_{col}" for col in variables_poly.columns]
variables_poly.columns = variables_poly.columns.str.replace(" ", "__")
variables_exogenas = pd.concat([variables_exogenas, variables_poly], axis=1)
variables_exogenas.head(3)

,Date,mes,semana_anyo,dia_semana,day_of_Year,hora_dia,hora_amanecer,hora_anochecer,horas_luz_solar,es_de_dia,...,poly_hora_anochecer_seno__hora_anochecer_coseno,poly_hora_anochecer_seno__hora_anochecer_coseno,poly_hora_anochecer_seno__horas_luz_solar,poly_hora_anochecer_seno__es_de_dia,poly_hora_anochecer_coseno__hora_anochecer_coseno,poly_hora_anochecer_coseno__horas_luz_solar,poly_hora_anochecer_coseno__es_de_dia,poly_hora_anochecer_coseno__horas_luz_solar,poly_hora_anochecer_coseno__es_de_dia,poly_horas_luz_solar__es_de_dia
0,2023-01-08,1,1,7,8,1,7,16,9,0,...,0.433013,0.433013,-7.794229,-0.0,0.250000,-4.50000,-0.0,-4.50000,-0.0,0.0
1,2023-01-09,1,2,1,9,1,7,16,9,0,...,0.433013,0.433013,-7.794229,-0.0,0.250000,-4.50000,-0.0,-4.50000,-0.0,0.0
2,2023-01-10,1,2,2,10,1,7,17,10,0,...,0.250000,0.250000,-9.659258,-0.0,0.066987,-2.58819,-0.0,-2.58819,-0.0,0.0


In [423]:
# Transformación con codificación one-hot
# ==============================================================================
one_hot_encoder = make_column_transformer(
    (
        OneHotEncoder(sparse_output=False, drop='if_binary'),
        make_column_selector(dtype_include=['category', 'object']),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")  

In [424]:
# Crear un forecaster con un transformer para las variables exógenas
# ==============================================================================
forecaster = ForecasterAutoreg(
    regressor        = LGBMRegressor(random_state=15926, verbose=-1),
    lags             = 72,
    transformer_exog = one_hot_encoder
)

In [426]:
# Mostrar matrices de entrenamiento
# ==============================================================================
exog_cols = ['weather']        
X_train, y_train = forecaster.create_train_X_y(
    y    = y_train_soles,
    exog = y_train_soles
)
X_train.head(3)

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,...,lag_64,lag_65,lag_66,lag_67,lag_68,lag_69,lag_70,lag_71,lag_72,Soles_Withdrawn
72,29328.0,25758.0,29878.0,31878.0,33913.0,13815.0,41533.0,23325.0,49460.0,43320.0,...,33132.0,30592.0,17094.0,33692.0,30505.0,53289.0,15459.0,29123.0,20475.0,14043
73,14043.0,29328.0,25758.0,29878.0,31878.0,33913.0,13815.0,41533.0,23325.0,49460.0,...,29898.0,33132.0,30592.0,17094.0,33692.0,30505.0,53289.0,15459.0,29123.0,22689
74,22689.0,14043.0,29328.0,25758.0,29878.0,31878.0,33913.0,13815.0,41533.0,23325.0,...,36168.0,29898.0,33132.0,30592.0,17094.0,33692.0,30505.0,53289.0,15459.0,50294


In [429]:
# Transformación: codificación ordinal + conversión a tipo "category"
# ==============================================================================
pipeline_categorical = make_pipeline(
    OrdinalEncoder(
        dtype=int,
        handle_unknown="use_encoded_value",
        unknown_value=-1,
        encoded_missing_value=-1
    ),
    FunctionTransformer(
        func=lambda x: x.astype('category'),
        feature_names_out= 'one-to-one'
    )
)

transformer_exog = make_column_transformer(
    (
        pipeline_categorical,
        make_column_selector(dtype_include=['category', 'object']),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

In [430]:
# Crear un forecaster con detección automática de variables categóricas (LGBMRegressor)
# ==============================================================================
forecaster = ForecasterAutoreg(
    regressor        = LGBMRegressor(random_state=15926, verbose=-1),
    lags             = 72,
    transformer_exog = transformer_exog,
    fit_kwargs       = {"categorical_feature": "auto"}
)

In [431]:
# Selección de variables exógenas a incluir en el modelo
# ==============================================================================
exog_cols = []
# Columnas que terminan con _seno o _coseno son seleccionadas
exog_cols.extend(variables_exogenas.filter(regex='_seno$|_coseno$').columns.tolist())
# Columnas que empiezan con tem_ son seleccionadas
exog_cols.extend(variables_exogenas.filter(regex='^temp_.*').columns.tolist())
# Columnas que empiezan con holiday_ son seleccionadas
exog_cols.extend(variables_exogenas.filter(regex='^holiday_.*').columns.tolist())
exog_cols.extend(['temp', 'holiday', 'weather'])

variables_exogenas = variables_exogenas.filter(exog_cols, axis=1)

ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
# Combinar variables exógenas y target en el mismo dataframe
# ==============================================================================
datos = datos[['users', 'weather']].merge(
    variables_exogenas,
    left_index=True,
    right_index=True,
    how='left'
)

# Debido a la creación de medias móviles, hay valores NaN al principio de la serie.
# Debido a holiday_dia_siguiente hay valores NaN al final de la serie.
# Las columnas numéricas se convierten a float32.
datos = datos.dropna()
datos = datos.astype({col: np.float32 for col in datos.select_dtypes("number").columns})
datos_train = datos.loc[: fin_train, :].copy()
datos_val   = datos.loc[fin_train:fin_validacion, :].copy()
datos_test  = datos.loc[fin_validacion:, :].copy()

In [436]:
# Búsqueda de hiperparámetros
# ==============================================================================
forecaster = ForecasterAutoreg(
    regressor        = LGBMRegressor(random_state=15926, verbose=-1),
    lags             = 72,
    transformer_exog = transformer_exog,
    fit_kwargs       = {"categorical_feature": "auto"}
)

# Lags grid
lags_grid = [48, 72, (1, 2, 3, 23, 24, 25, 167, 168, 169)]

# Espacio de búsqueda de hiperparámetros
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 25, 500),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 0.5),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1, step=0.1),
        'max_bin'         : trial.suggest_int('max_bin', 50, 250, step=25),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1, step=0.1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1, step=0.1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = y_soles,
    exog               = y_soles,
    search_space       = search_space,
    steps              = 36,
    refit              = False,
    metric             = 'mean_absolute_error',
    initial_train_size = len(y_train_soles),
    fixed_train_size   = False,
    n_trials           = 20,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)
best_params = results_search['params'].iat[0]

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'n_estimators': 1100, 'max_depth': 9, 'min_data_in_leaf': 25, 'learning_rate': 0.10558761933730798, 'feature_fraction': 0.9, 'max_bin': 75, 'reg_alpha': 0.9, 'reg_lambda': 0.30000000000000004}
  Backtesting metric: 384.61700112531486



In [439]:
# Backtesting en los datos de test incluyendo las variables exógenas
# ==============================================================================
metrica, predicciones = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = y_soles,
    exog               = y_soles,
    steps              = 36,
    metric             = 'mean_absolute_error',
    initial_train_size = len(y_train_soles),
    refit              = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)
metrica

  0%|          | 0/20 [00:00<?, ?it/s]

,mean_absolute_error
0,384.617001


In [441]:
# Gráfico predicciones vs valor real
# ======================================================================================
fig = go.Figure()
trace1 = go.Scatter(x=y_test_soles.index, y=y_test_soles, name="test", mode="lines")
trace2 = go.Scatter(x=predicciones.index, y=predicciones['pred'], name="prediction", mode="lines")
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
    title="Predicción vs valor real en los datos de test",
    xaxis_title="Fecha",
    yaxis_title="Withdrawn",
    width=800,
    height=350,
    margin=dict(l=20, r=20, t=35, b=20),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="left",
        x=0.001
    )
)
fig.show()